<a href="https://colab.research.google.com/github/dave-does-data/llm-qa-rlhf/blob/main/llm_qa_plhf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!git clone https://github.com/dave-does-data/llm-qa-rlhf.git
!pip install -r llm-qa-rlhf/requirements.txt

In [21]:
#from datasets import load_dataset,concatenate_datasets
#dataset = load_dataset("databricks/databricks-dolly-15k")
#
#def process_data(dataset):
#  append_a = dataset.filter(lambda row: row["category"]=='open_qa')
#  append_b = dataset.filter(lambda row: row["category"]=='closed_qa')
#  append_c = dataset.filter(lambda row: row["category"]=='general_qa')
#  qa_dataset = concatenate_datasets([append_a, append_b, append_c]).shuffle(seed=42)
#  qa_dataset = qa_dataset.add_column("text",['### Question: ' +str(row['instruction']) + '### Answer: ' + str(row['response']) for row in qa_dataset])
#  return qa_dataset
#
#train = process_data(dataset['train'])
#train.to_csv('databricks-dolly-qa-subset-7k.csv')

In [26]:
!git clone https://github.com/lvwerra/trl

Cloning into 'trl'...
remote: Enumerating objects: 3759, done.
remote: Counting objects: 100% (1745/1745), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 3759 (delta 1428), reused 1382 (delta 1208), pack-reused 2014
Receiving objects: 100% (3759/3759), 5.70 MiB | 24.51 MiB/s, done.
Resolving deltas: 100% (2317/2317), done.


In [ ]:
!python trl/examples/scripts/sft_trainer.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name dave-does-data/databricks-dolly-qa-subset-7k \
    --load_in_4bit \
    --use_peft \
    --batch_size 4 \
    --gradient_accumulation_steps 2